In [ ]:
# Install required packages
!pip install accelerate bitsandbytes diffusers torch transformers pyexiv2 tenacity google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client

import os
import pyexiv2
from PIL import Image
import time
from tenacity import retry, stop_after_attempt, wait_exponential
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from diffusers import DiffusionPipeline
import torch
import json
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload

# Load the model
quantization_config = BitsAndBytesConfig(load_in_4bit=True)
tokenizer = AutoTokenizer.from_pretrained("/kaggle/input/gemma/transformers/1.1-2b-it/1/")
model = AutoModelForCausalLM.from_pretrained(
    "/kaggle/input/gemma/transformers/1.1-2b-it/1/",
    quantization_config=quantization_config
)
model.eval()

# Set up Stable Diffusion pipeline
import time
from diffusers import DiffusionPipeline
from requests.exceptions import ReadTimeout, ConnectionError

def load_pipeline_with_retries(model_id, retries=3, delay=5):
    for attempt in range(retries):
        try:
            pipeline = DiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
            return pipeline
        except (ReadTimeout, ConnectionError) as e:
            print(f"Attempt {attempt + 1} failed: {e}. Retrying in {delay} seconds...")
            time.sleep(delay)
    raise Exception(f"Failed to load the model {model_id} after {retries} attempts.")

# Usage
model_id = "stabilityai/stable-diffusion-xl-base-1.0"
pipeline = load_pipeline_with_retries(model_id)
if torch.cuda.is_available():
    pipeline.to("cuda")
    pipeline.enable_attention_slicing()
    model.to("cuda")
    print("Using CUDA")
else:
    print("Using CPU")

# Define helper functions
def generate_text(prompt, max_length=100):
    inputs = tokenizer(prompt, return_tensors="pt")
    if torch.cuda.is_available():
        inputs = inputs.to("cuda")
    
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=max_length, num_return_sequences=1, temperature=0.7, do_sample=True)
    
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

def clear_folder(folder_path):
    for file in os.listdir(folder_path):
        os.remove(os.path.join(folder_path, file))
    print("Folder cleared successfully.")

def generate_prompts(keywords, num_prompts):
    prompts = []
    keyword_string = ", ".join(keywords)
    base_prompt = f"Generate a detailed and creative image prompt based on these keywords: {keyword_string}. The prompt should be:"
    
    while len(prompts) < num_prompts:
        generated_text = generate_text(base_prompt, max_length=100)
        clean_prompt = generated_text.replace(base_prompt, "").strip()
        if clean_prompt and clean_prompt not in prompts:
            prompts.append(clean_prompt)
    
    return prompts

def generate_keywords(prompt):
    base_prompt = f"Generate 30 relevant, simple keywords for this image prompt, separated by commas: {prompt}\nKeywords:"
    generated_text = generate_text(base_prompt, max_length=200)
    keywords = generated_text.replace(base_prompt, "").strip().split(',')
    keywords = [keyword.strip() for keyword in keywords if keyword.strip()]
    keywords.append('_ai_generated')
    return keywords

def generate_title(prompt):
    base_prompt = f"Generate a simple, catchy, and detailed title (less than 100 characters) for an image based on this prompt: {prompt}\nTitle:"
    generated_text = generate_text(base_prompt, max_length=100)
    return generated_text.replace(base_prompt, "").strip()

def upscale_and_save_image(image, jpg_path, min_file_size=500*1024):
    width, height = image.size
    scale_factor = 4
    quality = 95

    while True:
        new_size = (width * scale_factor, height * scale_factor)
        upscaled_image = image.resize(new_size, resample=Image.BICUBIC)
        upscaled_image.save(jpg_path, format="JPEG", quality=quality)

        if os.path.getsize(jpg_path) >= min_file_size:
            break

        if scale_factor < 8:
            scale_factor += 1
        elif quality < 100:
            quality += 5
        else:
            print(f"Warning: Unable to reach {min_file_size/1024}KB for {jpg_path} even after significant upscaling.")
            break

    return upscaled_image

@retry(stop=stop_after_attempt(5), wait=wait_exponential(multiplier=1, min=4, max=10))
def generate_image(pipeline, prompt):
    return pipeline(prompt).images[0]

# Google Drive API setup
def authenticate_drive_service(credentials_file):
    credentials = service_account.Credentials.from_service_account_file(credentials_file)
    drive_service = build('drive', 'v3', credentials=credentials)
    return drive_service

def upload_file_to_drive(drive_service, file_path, folder_id):
    file_metadata = {'name': os.path.basename(file_path), 'parents': [folder_id]}
    media = MediaFileUpload(file_path, resumable=True)
    file = drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()
    return file.get('id')

# Main execution
if __name__ == "__main__":
    # Path to your service account JSON file
    credentials_file = "/kaggle/input/service-account-credentials/precise-braid-383214-b0cd17ce3d76.json"
    drive_service = authenticate_drive_service(credentials_file)

    # Public folder ID on Google Drive
    PUBLIC_FOLDER_ID = "1yfkEd7BjUdiAO0FuzTqxKQCyglHl_r6n"
    
    keywords = ["River landscapes", "River trekking", "Trekking", "Kayak", "River water", "Mountain river", "River rafting", "photo-realistic", "3d render"]
    
    output_folder = "/kaggle/working/generated_images"
    os.makedirs(output_folder, exist_ok=True)
    
    print("Clearing the folder...")
    clear_folder(output_folder)

    num_prompts = 100
    prompts = generate_prompts(keywords, num_prompts)
    print(f"Generated {len(prompts)} prompts:")
    for i, prompt in enumerate(prompts, 1):
        print(f"{i}. {prompt}")

    prompt_titles = {}

    for prompt in prompts[:]:
        prompt = prompt.replace('.', '')

        try:
            keywords = generate_keywords(prompt)
            title = generate_title(prompt)
            prompt_titles[prompt] = title

            print(f"Prompt: {prompt}")
            print(f"Title: {title}")

            for i in range(5):  # num_images
                try:
                    image = generate_image(pipeline, prompt)
                    temp_jpg_path = f"{output_folder}/image-{title}-{i}.jpg"
                    upscaled_image = upscale_and_save_image(image, temp_jpg_path)

                    # Add IPTC metadata
                    image_data = pyexiv2.Image(temp_jpg_path)
                    image_data.modify_iptc({
                        'Iptc.Application2.Keywords': keywords,
                        'Iptc.Application2.Headline': title
                    })
                    image_data.close()

                    # Upload to Google Drive
                    upload_file_to_drive(drive_service, temp_jpg_path, PUBLIC_FOLDER_ID)
                    print(f"Uploaded {temp_jpg_path} to Google Drive")

                except Exception as e:
                    print(f"Error generating/saving image {i+1} for prompt '{prompt}': {e}")
                    continue

        except Exception as e:
            print(f"Error processing prompt '{prompt}': {e}")
            time.sleep(30)
            continue

        time.sleep(5)

    print("Image generation and upload process completed.")


`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

model_index.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/2.78G [00:00<?, ?B/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

Uploaded /kaggle/working/generated_images/image-**Reflections of Majesty**-4.jpg to Google Drive
Prompt: **A breathtaking photorealistic 3D render of a majestic mountain river flowing through a rugged and mountainous landscape The image captures the captivating energy and power of the river, showcasing kayakers navigating through swirling rapids and through towering cliffs The scene is illuminated by the ethereal
Title: **Dancing Waters**


  0%|          | 0/50 [00:00<?, ?it/s]

Uploaded /kaggle/working/generated_images/image-**Dancing Waters**-0.jpg to Google Drive


  0%|          | 0/50 [00:00<?, ?it/s]